# Práctica 1 - Self-Organising Maps
## Preparación de entorno
#### Importar librerías de código

In [21]:
%reset
# from __future__ import division

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches as patches

%matplotlib inline

#### Dataset que se va a utilizar para el entrenamiento

In [22]:
# Código para obtener el Dataset que se va a usar en el entrenamiento
datos = np.random.randint(0, 255, (100, 3))

## SOM Setup
#### Variables definidas por el alumno

In [23]:
# Inicializa tamaño del mapa de Kohonen, número de iteraciones y learning rate
# Inicializa normalizar_datos dependiendo de si tienes que normalizar los datos o no
lado_mapa = 10
periodo = 100
learning_rate = 0.1
normalizar_datos = True

#### A partir de este punto solo hay cálculos. No se introducen más valores "a mano"

In [ ]:
# Normaliza los datos si fuese necesario dividiendo cada dato por el máximo en la matriz
if normalizar_datos:
    datos = datos/255

# Establece el numero de entradas del mapa y el número de datos que se van a usar para entrenar. 
# Utiliza una función que obtenga automáticamente los valores a partir del Dataset.
 
def patron_escogido(datos):
    valor = np.random.randint(0, 100)
    patron_elegido = datos[valor]
    num_entradas = patron_elegido.size
    return patron_elegido, num_entradas

patron_elegido, num_entradas = patron_escogido(datos)
print("El patron elegido es ",patron_elegido," y el numero de entradas es ",num_entradas)
# Calcula el vecindario inicial. Debe ser la mitad del lado del mapa de Kohonen
vecindario = lado_mapa // 2



    
# Crea una matriz de pesos con valores random entre 0 y 1. Usa la función random.random de la librería NumPy
#matriz_pesos = np.random.random((dimensiones de la matriz de pesos))
matriz_pesos = np.random.random((lado_mapa, lado_mapa, num_entradas))
print(matriz_pesos.shape)


El patron elegido es  [0.70196078 0.49019608 0.89019608]  y el numero de entradas es  3
[[[0.88195114 0.06458405 0.84820435]
  [0.18856209 0.81612384 0.98682931]
  [0.04039827 0.75463081 0.53988221]
  [0.91656793 0.38888634 0.21052149]
  [0.16808955 0.56432328 0.76785865]
  [0.71624767 0.87353481 0.09625227]
  [0.90631882 0.14137314 0.82147229]
  [0.0053746  0.43131789 0.20036477]
  [0.73614054 0.27715396 0.69476908]
  [0.87641451 0.86876555 0.00615081]]

 [[0.49513105 0.20074465 0.15651467]
  [0.95621074 0.37210437 0.8586705 ]
  [0.78553761 0.95538729 0.61208571]
  [0.02765565 0.87483601 0.5819111 ]
  [0.72960726 0.80140436 0.3951523 ]
  [0.77590257 0.41069747 0.63418971]
  [0.26630306 0.90485039 0.20618841]
  [0.92135492 0.99538956 0.206606  ]
  [0.36753541 0.16271335 0.43514937]
  [0.58954723 0.21339139 0.2397626 ]]

 [[0.4778773  0.97720352 0.58899777]
  [0.37338948 0.17764882 0.63263883]
  [0.80801566 0.08464339 0.82643914]
  [0.22897271 0.31861455 0.65989688]
  [0.36771005 0.2679

#### Funciones para entrenar/clasificar

In [25]:
# Función para encontrar la BMU
"""
   Encuentra la BMU para un patrón de entrada.
   Entradas: (patrón_de_entrada, matriz_de_pesos, número_de_entradas)
   Salidas:  (bmu, bmu_idx) tupla donde
               bmu: vector de pesos de la neurona ganadora
               bum_idx: coordenadas de la neurona ganadora
"""
def calcular_bmu(patron_entrada, m_pesos, m):

SyntaxError: incomplete input (3651684424.py, line 9)

In [ ]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta)
"""
   Calcula el Learning Rate (eta) que corresponde a la i-ésima presentación.
   Entradas: (learning_rate_inicial, iteracion, período)
   Salidas:  learning_rate para la iteración i

"""
def variacion_learning_rate(lr_inicial, i, n_iteraciones):

In [ ]:
# Función para calcular el descenso del vecindario (v)
"""
   Calcula el vecindario  (v) que corresponde a la i-ésima presentación.
   Entradas: (vecindario_inicial, iteracion, período)
   Salidas:  lvecindario para la iteración i

"""
def variacion_vecindario(vecindario_inicial, i, n_iteraciones):

In [ ]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta) en función de la distancia a la BMU
"""
   Calcula la amortiguación de eta en función de la distancia en el mapa entre una neurona y la BMU.
   Entradas: (distancia_BMU, vecindario_actual)
   Salidas:  amortiguación para la iteración

"""
def decay(distancia_BMU, vecindario_actual):
    return np.exp(-distancia_BMU / (2* (vecindario_actual**2)))

#### Funciones para dibujar la salida de la red

In [ ]:
# Función para pintar una matriz de valores como colores RGB
def pintar_mapa(matriz_valores):
    fig = plt.figure()
    
    # Establece ejes
    ax = fig.add_subplot(111, aspect='equal')
    ax.set_xlim((0, matriz_pesos.shape[0]+1))
    ax.set_ylim((0, matriz_pesos.shape[1]+1))
    ax.set_title('Self-Organising Map después de %d iteraciones' % periodo)

    # Dibuja los rectángulos de color RGB
    for x in range(1, matriz_valores.shape[0] + 1):
        for y in range(1, matriz_valores.shape[1] + 1):
            ax.add_patch(patches.Rectangle((x-0.5, y-0.5), 1, 1,
                         facecolor=matriz_valores[x-1,y-1,:],
                         edgecolor='none'))
    plt.show()

## SOM Entrenamiento

In [ ]:
# Entrena la red con el dataset de entrenamiento



## SOM Clasificación

In [ ]:
# Clasifica los patrones de entrenamiento con la matriz de pesos recién entrenada



## SOM Prueba

In [ ]:
# Clasifica nuevos patrones

